In [1]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [7]:
# Training data is 100 points in [0,1] inclusive regularly spaced
train_x = torch.linspace(0, 1, 100)
# True function is sin(2*pi*x) with Gaussian noise
train_y = torch.sin(train_x * (2 * math.pi)) + torch.randn(train_x.size()) * math.sqrt(0.04)

In [2]:
# We will use the simplest form of GP model, exact inference
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(train_x, train_y, likelihood)

NameError: name 'train_x' is not defined

In [9]:
# this is for running the notebook in our testing framework
import os
smoke_test = ('CI' in os.environ)
training_iter = 2 if smoke_test else 50


# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(train_x)
    # Calc loss and backprop gradients
    loss = -mll(output, train_y)
    loss.backward()
    print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
        i + 1, training_iter, loss.item(),
        model.covar_module.base_kernel.lengthscale.item(),
        model.likelihood.noise.item()
    ))
    optimizer.step()

Iter 1/50 - Loss: 0.932   lengthscale: 0.693   noise: 0.693
Iter 2/50 - Loss: 0.901   lengthscale: 0.644   noise: 0.644
Iter 3/50 - Loss: 0.866   lengthscale: 0.598   noise: 0.598
Iter 4/50 - Loss: 0.828   lengthscale: 0.555   noise: 0.554
Iter 5/50 - Loss: 0.785   lengthscale: 0.514   noise: 0.513
Iter 6/50 - Loss: 0.737   lengthscale: 0.475   noise: 0.474
Iter 7/50 - Loss: 0.685   lengthscale: 0.439   noise: 0.437
Iter 8/50 - Loss: 0.633   lengthscale: 0.404   noise: 0.402
Iter 9/50 - Loss: 0.583   lengthscale: 0.372   noise: 0.369
Iter 10/50 - Loss: 0.537   lengthscale: 0.342   noise: 0.339
Iter 11/50 - Loss: 0.495   lengthscale: 0.315   noise: 0.310
Iter 12/50 - Loss: 0.456   lengthscale: 0.291   noise: 0.284
Iter 13/50 - Loss: 0.419   lengthscale: 0.272   noise: 0.259
Iter 14/50 - Loss: 0.384   lengthscale: 0.256   noise: 0.236
Iter 15/50 - Loss: 0.349   lengthscale: 0.243   noise: 0.215
Iter 16/50 - Loss: 0.315   lengthscale: 0.232   noise: 0.196
Iter 17/50 - Loss: 0.280   length

In [12]:
test_x = torch.linspace(0, 1, 20)
f_preds = model(test_x)
y_preds = likelihood(model(test_x))

f_mean = f_preds.mean
f_var = f_preds.variance
f_covar = f_preds.covariance_matrix
f_samples = f_preds.sample(sample_shape=torch.Size(1000,))

RuntimeError: You must train on the training inputs!